In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import datetime
from collections import OrderedDict

In [3]:
df = pd.read_excel("/content/drive/MyDrive/output_anonomization_no_names_only_english_version2.xlsx")

In [ ]:
df.shape

(167972, 36)

In [4]:
df.columns

Index(['Unnamed: 0', 'number', 'opened_by', 'short_description', 'priority',
       'assignment_group', 'assigned_to', 'u_main_category_reporting',
       'u_subcategory_1_reporting', 'u_subcategory_2_reporting',
       'u_subcategory_3_reporting', 'u_resolver_group', 'u_resolver',
       'u_first_assignment_group', 'u_country', 'sys_updated_on',
       'sys_updated_by', 'sys_mod_count', 'u_vip', 'u_type', 'u_region',
       'contact_type', 'location', 'u_resolved', 'closed_at',
       'u_assignment_group_history', 'work_notes', 'comments', 'close_notes',
       'u_assignee_history', 'merged', 'merged_string', 'business_duration',
       'u_reassignment_count_assigne', 'reassignment_count', 'cleaned_string',
       'isEnglish'],
      dtype='object')

In [ ]:
filt = df['isEnglish'] == 1

In [ ]:
df_filt = df.loc[filt]

In [ ]:
df_filt.shape

(67171, 36)

In [5]:
user_comment_column = []
agent_comment_column = []
problem_column = []
solution_column = []

In [6]:
for index, row in df.iterrows():
    text = str(row['comments']) + '\n' + str(row['work_notes'])
    history = str(row['u_assignee_history'])
    agents = history.split(',')
    split_list = text.split('\n')
    users_comments = []
    agents_comments = []
    comment = ""
    comments = []

    check = False
    for t in split_list:
        if ("Additional comments" in t or "Work notes" in t) and not check:
            check = True
        elif ("Additional comments" in t or "Work notes" in t) and check:
            comments.append(comment)
            comment = ""
        if check:
            comment += t + '\n'
    comments.append(comment)

    

    for c in comments:
        agent_check = False
        for agent in agents:
            if ("Additional comments" in c and agent in c[:c.find("Additional comments")]) or ("Work notes" in c and agent in c[:c.find("Work notes")]):
                x = "Issue description"
                y = "Issue(s)"
                z = "Short Description"
                if x in c or y in c or z in c:
                    if "My action taken" in c:
                        agents_comments.append(c)
                        agent_check = True
                        if x in c:
                            if x not in str(row["short_description"]):
                                users_comments.append(c[c.find(x):].split('\n')[0])

                        if y in c:
                            if y not in str(row["short_description"]):
                                users_comments.append(c[c.find(y):].split('\n')[0])

                        if z in c:
                            if z not in str(row["short_description"]):
                                users_comments.append(c[c.find(z):].split('\n')[0])
                else:
                    agents_comments.append(c)
                    agent_check = True
                break

    if not agent_check:
        users_comments.append(c)

    users_text = '\n'.join(users_comments)
    agents_text = '\n'.join(agents_comments)
    user_comment_column.append(users_text)
    agent_comment_column.append(agents_text)

    problem = str(row['short_description']) + '\n' + users_text
    solution = agents_text + '\n' + str(row['close_notes'])
    problem_column.append(problem)
    solution_column.append(solution)

    if index % 1000 == 0:
          print(index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [7]:
df['user_comment'] = user_comment_column
df['agent_comment'] = agent_comment_column
df['problem'] = problem_column
df['solution'] = solution_column

In [8]:
# TEST

df.loc[5, "solution"]

'2021-01-01 05:19:34 - [name_1] (Work notes)\n+6046325427\n\n\n2021-01-01 05:17:58 - [name_1] (Work notes)\nService Request has been created successfully. \n \nSoftware Assignment was successful \nDevice: MY02-C0032 Software: I_CheckPoint_EndpointSecurity_82.20_EN-US_x32_01  \n \nWho called : [name_4825] \nExt Number :   \nCall time :   (UTC+8) \nComputer [name_13918] :  MY02-C0032 \nLocations : BMI MY \nShort Description :  [name_11297] EndPoint | Unable to login \n \nIssue(s) : [name_7814] called on behalf of her. She cannot login via CheckPoint. \n \nMy action taken : I went to [name_7812] and pushed software.\n\n\n\n2021-01-01 05:18:30 - [name_1] (Close notes (Customer visible))\nDear User, \n \nThank You for your call.  \nThis ticket is the report of what we have assisted you by resolving your issue.   \n \nThank you and have a nice day.\n\n'

In [9]:
sorted_solution_column = []

In [10]:
for index, row in df.iterrows():
    text = str(row['solution'])
    split_list = text.split('\n')

    comment = ""
    comments = {}
    date = None
    check = False

    for t in split_list:
        if bool(x := re.search('^[\d]{4}-[\d]{2}-[\d]{2} [\d]{2}:[\d]{2}:[\d]{2}', t)) and not check:
            check = True
            date = x.group(0)
        elif bool(y := re.search('^[\d]{4}-[\d]{2}-[\d]{2} [\d]{2}:[\d]{2}:[\d]{2}', t)) and check:
            try:
                comments[date].append(comment)
            except:
                comments[date] = [comment]
            comment = ""
            date = y.group(0)

        if check:
            comment += t + '\n'

    try:
        comments[date].append(comment)
    except:
        comments[date] = [comment]

    try:
        ordered = OrderedDict(sorted(comments.items(), key=lambda x: datetime.datetime.strptime(x[0], '%Y-%m-%d %H:%M:%S'), reverse=False))
    except:
        ordered = {}

    ordered_text = ""
    for val in ordered.values():
        for val_text in val:
            ordered_text += val_text

    sorted_solution_column.append(ordered_text)

    if index % 1000 == 0:
          print(index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [11]:
df['solution_sorted'] = sorted_solution_column

In [12]:
df.to_excel('/content/drive/MyDrive/output_user_agent_no_names.xlsx')